In [ ]:
import pandas as pd

In [ ]:
data_tumor_1dpi = pd.read_csv("../../app/tumor_volumes_1dpi.csv")
data_tumor_5dpi = pd.read_csv("../../app/tumor_volumes_5dpi.csv")
data_tumor_5dpi.columns

In [ ]:
data_macrophage = pd.read_csv("../../app/macrophage_props_1dpi_and_5dpi.csv")
data_macrophage.columns

In [ ]:
t_1dpi_start = 26, 27

In [ ]:
limits_round = [0.6, 1.0]
limits_remified = [0.0, 0.35]

time_intervals_macro = {  "t_1dpi_start": [24,26],
                    "t_1dpi_mid": [30,32],
                    "t_1dpi_end": [37,39],
                    "t_5dpi_start": [114,115],
                    "t_5dpi_end":  [117,118]} #

time_intervals_tumor = { "t_1dpi": [24,39],
                         "t_5dpi": [114,118]
                       } #

In [ ]:
data_macrophage.columns

In [ ]:
data_macrophage.groupby(["fish_id","time_point"]).count().std()[0]

In [ ]:
data_round = data_macrophage[data_macrophage["circularity"] >= limits_round[0]]
data_remified = data_macrophage[data_macrophage["circularity"] <=  limits_remified[1]]

In [ ]:
matrix_cols = ['feature'] + list(data_macrophage['cancer_cells'].unique())
#matrix_cols = matrix_cols.append(data.columns)
print(matrix_cols)
data_matrix =  pd.DataFrame(columns = matrix_cols)
data_matrix

In [ ]:
len(data_matrix.index)

In [ ]:
def add_feature(data, data_matrix, index, feature, macrophage_type, time_interval):
    #index = len(data_cluster.index)
    
    t_start = time_intervals_macro[time_interval][0]
    t_end = time_intervals_macro[time_interval][1]
    
    for cancer_cell in data['cancer_cells'].unique():
        data_cancer_cell = data[data['cancer_cells']==cancer_cell]
        data_matrix.at[index ,"feature"] = "mean_%s_%s_%s" % (feature, macrophage_type,time_interval)
        data_cancer_cell_start = data_cancer_cell[data_cancer_cell['time_in_h'] >= t_start]
        data_cancer_cell_start =  data_cancer_cell_start[ data_cancer_cell_start['time_in_h'] <= t_end]
        if feature == "count":
            data_matrix.at[index,cancer_cell] = data_cancer_cell_start.groupby(["fish_id","time_point"]).count().mean()[0]
        else:
            data_matrix.at[index,cancer_cell] = data_cancer_cell_start[feature].mean()
        
    
    index += 1
    
    for cancer_cell in data['cancer_cells'].unique():
        data_cancer_cell = data[data['cancer_cells']==cancer_cell]
        data_matrix.at[index ,"feature"] = "std_%s_%s_%s" % (feature,macrophage_type,time_interval)
        data_cancer_cell_start = data_cancer_cell[data_cancer_cell['time_in_h'] >= t_start]
        data_cancer_cell_start =  data_cancer_cell_start[ data_cancer_cell_start['time_in_h'] <= t_end]
        if feature == "count":
            data_matrix.at[index,cancer_cell] = data_cancer_cell_start.groupby(["fish_id","time_point"]).count().std()[0]
        else:
            data_matrix.at[index,cancer_cell] = data_cancer_cell_start[feature].std()
    
    index += 1
      
    return data_matrix, index
    #index += 1
    

In [ ]:
def add_feature_tumor(data, data_matrix, index, feature, time_interval):
    #index = len(data_cluster.index)
    
    t_start = time_intervals_tumor[time_interval][0]
    t_end = time_intervals_tumor[time_interval][1]
    
    for cancer_cell in data['cancer_cells'].unique():
        data_cancer_cell = data[data['cancer_cells']==cancer_cell]
        data_matrix.at[index ,"feature"] = "mean_%s_%s" % (feature, time_interval)
        data_cancer_cell_start = data_cancer_cell[data_cancer_cell['time_in_h'] >= t_start]
        data_cancer_cell_start =  data_cancer_cell_start[ data_cancer_cell_start['time_in_h'] <= t_end]
        data_matrix.at[index,cancer_cell] = data_cancer_cell_start[feature].mean()
        
    
    index += 1
    
    for cancer_cell in data['cancer_cells'].unique():
        data_cancer_cell = data[data['cancer_cells']==cancer_cell]
        data_matrix.at[index ,"feature"] = "std_%s_%s" % (feature,time_interval)
        data_cancer_cell_start = data_cancer_cell[data_cancer_cell['time_in_h'] >= t_start]
        data_cancer_cell_start =  data_cancer_cell_start[ data_cancer_cell_start['time_in_h'] <= t_end]
        data_matrix.at[index,cancer_cell] = data_cancer_cell_start[feature].std()
    
    index += 1
      
    return data_matrix, index
    #index += 1

In [ ]:
index = 0
data_matrix, index = add_feature(data_round, data_matrix, index, "count", "round", "t_1dpi_start")
data_matrix, index = add_feature(data_round, data_matrix, index, "count", "round", "t_1dpi_mid")
data_matrix, index = add_feature(data_round, data_matrix, index, "count", "round", "t_1dpi_end")
data_matrix, index = add_feature(data_round, data_matrix, index, "count", "round", "t_5dpi_start")
data_matrix, index = add_feature(data_round, data_matrix, index, "count", "round", "t_5dpi_end")

data_matrix, index = add_feature(data_remified, data_matrix, index, "count", "remified", "t_1dpi_start")
data_matrix, index = add_feature(data_remified, data_matrix, index, "count", "remified", "t_1dpi_mid")
data_matrix, index = add_feature(data_remified, data_matrix, index, "count", "remified", "t_1dpi_end")
data_matrix, index = add_feature(data_remified, data_matrix, index, "count", "remified", "t_5dpi_start")
data_matrix, index = add_feature(data_remified, data_matrix, index, "count", "remified", "t_5dpi_end")

data_matrix, index = add_feature(data_round, data_matrix, index, "area_mum2", "round", "t_1dpi_start")
data_matrix, index = add_feature(data_round, data_matrix, index, "area_mum2", "round", "t_1dpi_mid")
data_matrix, index = add_feature(data_round, data_matrix, index, "area_mum2", "round", "t_1dpi_end")
data_matrix, index = add_feature(data_round, data_matrix, index, "area_mum2", "round", "t_5dpi_start")
data_matrix, index = add_feature(data_round, data_matrix, index, "area_mum2", "round", "t_5dpi_end")

data_matrix, index = add_feature(data_remified, data_matrix, index, "area_mum2", "remified", "t_1dpi_start")
data_matrix, index = add_feature(data_remified, data_matrix, index, "area_mum2", "remified", "t_1dpi_mid")
data_matrix, index = add_feature(data_remified, data_matrix, index, "area_mum2", "remified", "t_1dpi_end")
data_matrix, index = add_feature(data_remified, data_matrix, index, "area_mum2", "remified", "t_5dpi_start")
data_matrix, index = add_feature(data_remified, data_matrix, index, "area_mum2", "remified", "t_5dpi_end")

data_matrix, index = add_feature(data_round, data_matrix, index, "perimeter_mum", "round", "t_1dpi_start")
data_matrix, index = add_feature(data_round, data_matrix, index, "perimeter_mum", "round", "t_1dpi_mid")
data_matrix, index = add_feature(data_round, data_matrix, index, "perimeter_mum", "round", "t_1dpi_end")
data_matrix, index = add_feature(data_round, data_matrix, index, "perimeter_mum", "round", "t_5dpi_start")
data_matrix, index = add_feature(data_round, data_matrix, index, "perimeter_mum", "round", "t_5dpi_end")

data_matrix, index = add_feature(data_remified, data_matrix, index, "perimeter_mum", "remified", "t_1dpi_start")
data_matrix, index = add_feature(data_remified, data_matrix, index, "perimeter_mum", "remified", "t_1dpi_mid")
data_matrix, index = add_feature(data_remified, data_matrix, index, "perimeter_mum", "remified", "t_1dpi_end")
data_matrix, index = add_feature(data_remified, data_matrix, index, "perimeter_mum", "remified", "t_5dpi_start")
data_matrix, index = add_feature(data_remified, data_matrix, index, "perimeter_mum", "remified", "t_5dpi_end")

data_matrix, index = add_feature(data_round, data_matrix, index, "eccentricity", "round", "t_1dpi_start")
data_matrix, index = add_feature(data_round, data_matrix, index, "eccentricity", "round", "t_1dpi_mid")
data_matrix, index = add_feature(data_round, data_matrix, index, "eccentricity", "round", "t_1dpi_end")
data_matrix, index = add_feature(data_round, data_matrix, index, "eccentricity", "round", "t_5dpi_start")
data_matrix, index = add_feature(data_round, data_matrix, index, "eccentricity", "round", "t_5dpi_end")

data_matrix, index = add_feature(data_remified, data_matrix, index, "eccentricity", "remified", "t_1dpi_start")
data_matrix, index = add_feature(data_remified, data_matrix, index, "eccentricity", "remified", "t_1dpi_mid")
data_matrix, index = add_feature(data_remified, data_matrix, index, "eccentricity", "remified", "t_1dpi_end")
data_matrix, index = add_feature(data_remified, data_matrix, index, "eccentricity", "remified", "t_5dpi_start")
data_matrix, index = add_feature(data_remified, data_matrix, index, "eccentricity", "remified", "t_5dpi_end")

data_matrix, index = add_feature(data_round, data_matrix, index, "circularity", "round", "t_1dpi_start")
data_matrix, index = add_feature(data_round, data_matrix, index, "circularity", "round", "t_1dpi_mid")
data_matrix, index = add_feature(data_round, data_matrix, index, "circularity", "round", "t_1dpi_end")
data_matrix, index = add_feature(data_round, data_matrix, index, "circularity", "round", "t_5dpi_start")
data_matrix, index = add_feature(data_round, data_matrix, index, "circularity", "round", "t_5dpi_end")

data_matrix, index = add_feature(data_remified, data_matrix, index, "circularity", "remified", "t_1dpi_start")
data_matrix, index = add_feature(data_remified, data_matrix, index, "circularity", "remified", "t_1dpi_mid")
data_matrix, index = add_feature(data_remified, data_matrix, index, "circularity", "remified", "t_1dpi_end")
data_matrix, index = add_feature(data_remified, data_matrix, index, "circularity", "remified", "t_5dpi_start")
data_matrix, index = add_feature(data_remified, data_matrix, index, "circularity", "remified", "t_5dpi_end")

data_matrix, index = add_feature(data_round, data_matrix, index, "distance_to_tumor", "round", "t_1dpi_start")
data_matrix, index = add_feature(data_round, data_matrix, index, "distance_to_tumor", "round", "t_1dpi_mid")
data_matrix, index = add_feature(data_round, data_matrix, index, "distance_to_tumor", "round", "t_1dpi_end")
data_matrix, index = add_feature(data_round, data_matrix, index, "distance_to_tumor", "round", "t_5dpi_start")
data_matrix, index = add_feature(data_round, data_matrix, index, "distance_to_tumor", "round", "t_5dpi_end")

data_matrix, index = add_feature(data_remified, data_matrix, index, "distance_to_tumor", "remified", "t_1dpi_start")
data_matrix, index = add_feature(data_remified, data_matrix, index, "distance_to_tumor", "remified", "t_1dpi_mid")
data_matrix, index = add_feature(data_remified, data_matrix, index, "distance_to_tumor", "remified", "t_1dpi_end")
data_matrix, index = add_feature(data_remified, data_matrix, index, "distance_to_tumor", "remified", "t_5dpi_start")
data_matrix, index = add_feature(data_remified, data_matrix, index, "distance_to_tumor", "remified", "t_5dpi_end")

data_matrix

# add tumor data

In [ ]:
data_matrix, index = add_feature_tumor(data_tumor_1dpi, data_matrix, index, "tumor_volume_um", "t_1dpi")
data_matrix, index = add_feature_tumor(data_tumor_5dpi, data_matrix, index, "tumor_volume_um", "t_5dpi")


#data_matrix, index = add_feature_tumor(data_tumor_1dpi, data_matrix, index, "macrophage_mean_distance", "t_1dpi_start")
#data_matrix, index = add_feature_tumor(data_tumor_1dpi, data_matrix, index, "macrophage_mean_distance", "t_1dpi_mid")
#data_matrix, index = add_feature_tumor(data_tumor_1dpi, data_matrix, index, "macrophage_mean_distance", "t_1dpi_end")
#data_matrix, index = add_feature_tumor(data_tumor_5dpi, data_matrix, index, "macrophage_mean_distance", "t_5dpi_start")
#data_matrix, index = add_feature_tumor(data_tumor_5dpi, data_matrix, index, "macrophage_mean_distance", "t_5dpi_end")
data_matrix

In [ ]:
data_matrix.to_csv("tumor_macrophage_data_matrix_for_scRNA_seq_mean_std.csv", index=False)

In [ ]:
select = ["mean" in row["feature"] for index, row in data_matrix.iterrows()]

In [ ]:
select

In [ ]:
data_matrix_mean = data_matrix[select]
data_matrix_mean.to_csv("tumor_macrophage_data_matrix_for_scRNA_seq.csv", index=False)